project/
├── gan_training.py          # Main script to train the GAN
├── generator.py             # Generator network definition
├── discriminator.py         # Discriminator network definition
├── dataset.py               # Dataset loader
├── metrics.py               # Metrics for evaluation (e.g., FID)
├── training_config.json     # Configuration for hyperparameters
└── utils.py                 # Helper functions (e.g., gradient penalty)


```
# This is formatted as code
```



In [1]:
# Install TensorFlow (if not already installed)
!pip install tensorflow

# Import required libraries
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [2]:
local_folder_path = r"C:\Users\MTechS_h\Desktop\Mini Project\ecg_data"


In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [4]:
pip install tensorflow-gpu


Note: you may need to restart the kernel to use updated packages.


In [6]:
import tensorflow as tf

# Set memory growth on GPU before any operations
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except RuntimeError as e:
        print(e)


In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Constants
IMG_HEIGHT, IMG_WIDTH = 64, 64
BUFFER_SIZE = 60000
BATCH_SIZE = 32  # Reduce batch size to fit in memory
EPOCHS = 50
NOISE_DIM = 100
NUM_EXAMPLES_TO_GENERATE = 16
SEED = 42

# Set random seeds for reproducibility
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Function to load and preprocess data
def load_and_preprocess_data(local_folder_path):
    print(f"Loading dataset from: {local_folder_path}")
    
    # List files in the dataset directory
    files_in_directory = os.listdir(local_folder_path)
    print(f"Files in directory: {files_in_directory}")
    
    images = []
    
    # Loop through each file in the directory and check if it is a valid PNG image
    for img_name in files_in_directory:
        img_path = os.path.join(local_folder_path, img_name)
        
        try:
            # Load and preprocess the image (resize and normalize)
            img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img = image.img_to_array(img)
            img = (img - 127.5) / 127.5  # Normalize to [-1, 1]
            images.append(img)
        except Exception as e:
            print(f"Skipping file {img_name} due to error: {e}")
    
    if not images:
        raise ValueError("No valid images found in the dataset.")
    
    return np.array(images)

# Function to build the generator model
def build_generator():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(8*8*256, use_bias=False, input_shape=(NOISE_DIM,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),

        tf.keras.layers.Reshape((8, 8, 256)),

        tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),

        tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),

        tf.keras.layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Function to build the discriminator model
def build_discriminator():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[IMG_HEIGHT, IMG_WIDTH, 3]),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)
    ])
    return model

# Define loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=build_generator(),
                                 discriminator=build_discriminator())

# Function to generate and save images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i] * 127.5 + 127.5).numpy().astype("uint8"))
        plt.axis('off')

    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.show()

# Training step function
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = build_generator()(noise, training=True)

        real_output = build_discriminator()(images, training=True)
        fake_output = build_discriminator()(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, build_generator().trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, build_discriminator().trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, build_generator().trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, build_discriminator().trainable_variables))

    return gen_loss, disc_loss

# Training function
def train(dataset, epochs):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        # Save the model every 5 epochs
        if (epoch + 1) % 5 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        # Generate and save images
        generate_and_save_images(build_generator(), epoch + 1, seed)

        print(f"Generator loss: {gen_loss}, Discriminator loss: {disc_loss}")

# Main function to load the data, create dataset, and start training
def main():
    local_folder_path = r"C:\Users\MTechS_h\Desktop\Mini Project\ecg_data\ECG_Image_data"

    # Load the dataset from the folder
    dataset = load_and_preprocess_data(local_folder_path)
    print(f"Loaded dataset with shape: {dataset.shape}")

    # Create TensorFlow Dataset
    dataset = tf.data.Dataset.from_tensor_slices(dataset).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

    # Set device to CPU to avoid GPU-related issues
    with tf.device('/CPU:0'):
        # Start the training process
        train(dataset, EPOCHS)

if __name__ == "__main__":
    main()
